In [171]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
import matplotlib 
import matplotlib.pyplot as plt
import math
from math import pi
import time
%matplotlib inline

In [172]:
#init variable
data = np.zeros([200,8])
train = np.zeros([140,8])
test = np.zeros([60,8])

In [173]:
#read file and split data as training and testing
def read(filepath):
    file = open(filepath)
    line = file.readline()
    n = 0
    while(line):
        line = file.readline()
        row = line.split()
        if(row!=[]):
            if(n<=139):
                for i in range(8):
                    train[n,i]=row[i]
                    data[n,i]=row[i]
            if(n>139):
                for i in range(8):
                    test[n-140,i]=row[i]
                    data[n,i]=row[i]
            n=n+1
    file.close()
    return data,train,test
#calculate mean
def mean(number):
    return sum(number)/len(number)
#calculate standard deviation for each feature of each class
def stdev(number):
    st = mean(number)
    sum = 0
    for n in range(len(number)):
        sum=sum + (number[n]-st)**2
    return math.sqrt(sum/len(number))

In [174]:
#seperate data to two different class for training
read("HW5_dataset.txt")
n0 = 0  #number of class 0
n1 = 0  #number of class 1
for label in train[:,0]:
    if label==0:
        n0 = n0+1
    if label==1:
        n1 = n1+1
s0 = np.zeros([n0,7])
s1 = np.zeros([n1,7])
s00 = 0
s11 = 0
for i in range(140):
    if(train[i,0]==0):
        s0[s00,:]=train[i,1:8]
        s00=s00+1
    if(train[i,0]==1):
        s1[s11,:]=train[i,1:8]
        s11=s11+1
#training data species0 s0, species1 s1

#possiblity of each class in sample
s0poss = s00/(s00+s11)
s1poss = s11/(s00+s11)
print(s0poss,s1poss)

#calculate mu0 and mu1
mu0=np.zeros([7,1])
mu1=np.zeros([7,1])
for n in range(7):
    mu0[n,0]=mean(s0[:,n])
    mu1[n,0]=mean(s1[:,n])

0.5142857142857142 0.4857142857142857


In [175]:
#calculate sigma for class 0 and 1
#@parameter: sigma0, sigma1
sigma0 = np.zeros([7,7])
sigma1 = np.zeros([7,7])
for num in range(len(s0)):
    sigma0 = sigma0 + (s0[num:num+1,:].T-mu0)@(s0[num:num+1,:].T-mu0).T
for num in range(len(s1)):
    sigma1 = sigma1 + (s1[num:num+1,:].T-mu1)@(s1[num:num+1,:].T-mu1).T
sigma0=sigma0/len(s0)
sigma1=sigma1/len(s1)

In [176]:
def determine_class(x):
    x.shape=(7,1)
    poss0=0
    poss1=0
    g0 = -1/2*(x-mu0).T@inv(sigma0)@(x-mu0)-1/2*math.log(det(-sigma0))-7/2*math.log(2*pi)+math.log(s0poss)
    g1 = -1/2*(x-mu1).T@inv(sigma1)@(x-mu1)-1/2*math.log(det(sigma1))-7/2*math.log(2*pi)+math.log(s1poss)
    if g0>g1:
        return 0
    else:
        return 1

In [177]:
#test accuracy on testing data
right=0
for i in range(60):
    label = test[i,0]
    answer = determine_class(test[i,1:8])
    if label==answer:
        right=right+1
print("Accuracy to predicte testing data:",right/60)

Accuracy to predicte testing data: 0.9833333333333333


In [178]:
#time needed for the entire dataset
start = time.clock()
right = 0
wrong_predict=[]
setnum=[]
for i in range(200):
    label = data[i,0]
    answer = determine_class(data[i,1:8])
    if label==answer:
        right=right+1
    if label!=answer:
        wrong_predict.append(data[i,0:8])
        setnum.append(i)
print("Time to classify the entire dataset",time.clock()-start,"s")
print("Number of correct prediction:",right,"Accuracy:",right/200)
#print(wrong_predict)
print("Line of wrong predicted data:",setnum)

Time to classify the entire dataset 0.031054000000001025 s
Number of correct prediction: 196 Accuracy: 0.98
Line of wrong predicted data: [12, 48, 83, 158]


In [179]:
#class confusion matrix
class0=0
class1=0
for i in range(200):
    if data[i,0]==0:
        class0=class0+1
    if data[i,0]==1:
        class1=class1+1
print(class0,class1)
for i in setnum:
    print(data[i,0])

100 100
0.0
0.0
0.0
0.0
